In [1]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
from scipy import ndimage
from skimage import img_as_ubyte

filedir = '/mnt/onetb/yuuka/lumen/'
savedir = filedir

def ResizeWithAspectRatio(image, width=None, height=None, inter=cv.INTER_AREA):
    dim = None
    (h, w) = image.shape[:2]

    if width is None and height is None:
        return image
    if width is None:
        r = height / float(h)
        dim = (int(w * r), height)
    else:
        r = width / float(w)
        dim = (width, int(h * r))

    return cv.resize(image, dim, interpolation=inter)

def affine_alignment(img1, img2):

    img1_gray = cv.cvtColor(img1, cv.COLOR_BGR2GRAY)
    img2_gray = cv.cvtColor(img2, cv.COLOR_BGR2GRAY)

    empty_array_1 = np.zeros(img1_gray.shape[:2], dtype=np.uint8)
    empty_array_2 = np.zeros(img2_gray.shape[:2], dtype=np.uint8)

    row1, col1 = img1_gray.shape
    row2, col2 = img2_gray.shape

    sift = cv.SIFT_create()

    kp1, d1 = sift.detectAndCompute(img1_gray, mask = None)
    kp2, d2 = sift.detectAndCompute(img2_gray, mask = None)

    bf = cv.BFMatcher()
    matches = bf.knnMatch(d1, d2, k = 2)

    ratio = 0.8

    good_matches = []
    for i, j in matches:
        if i.distance < j.distance * ratio:
            good_matches.append([i])

    source = np.float32([kp1[m[0].queryIdx].pt for m in good_matches]).reshape(-1,1,2)
    destination = np.float32([kp2[m[0].trainIdx].pt for m in good_matches]).reshape(-1,1,2)

    source = source
    destination = destination

    M, mask = cv.estimateAffine2D(source, destination)
    
    M[0,2] *= 8
    M[1,2] *= 8

    return M

#have N images
#find map between neighbouring images M from i+1 to i image
#apply maps using compositions
#can also have map aligned to previous image, this method sucks 
#deformations in previous images lead to even worse deformations in new images
#also it seems to be worse when doing masking on deformed images
#instead, store array of all transformations
#at the end, apply transformations to both images and masks
#save the masks, save the images

def load_img(index):
    
    img = cv.imread(filedir + 'Lumen_' + str(index) + '.tif')
    top, bottom, left, right = 47, 46, 193, 194
    img = cv.copyMakeBorder(img, top, bottom, left, right, borderType = cv.BORDER_CONSTANT)

    return img

def compose_affine_maps(M1, M2):
    #composition of M2 circ M1

    M1_new = np.zeros((3,3))
    M2_new = np.zeros((3,3))

    M1_new[0:2, :] = M1
    M1_new[2,2] = 1
    M2_new[0:2, :] = M2
    M2_new[2,2] = 1

    return np.matmul(M1_new, M2_new)[0:2, :]

start = 167
end = 238
        
prev_map = None
        
for i in range(start, end):
    
    if i == start:
        
        img1_full = load_img(i)
        img2_full = load_img(i + 1)

        img1 = cv.resize(img1_full, (950, 950))
        img2 = cv.resize(img2_full, (950, 950))
        
        M = affine_alignment(img2, img1)
        
        prev_map = M
        
        dst = cv.warpAffine(img2_full, M, (img2_full.shape[1], img2_full.shape[0]))
        cv.imwrite(savedir + f'lumen_affine_{i+1}.tif', dst)
        print(i)
        
    else:
        
        img1_full = img2_full
        img2_full = load_img(i + 1)

        img1 = cv.resize(img1_full, (950, 950))
        img2 = cv.resize(img2_full, (950, 950))

        M = affine_alignment(img2, img1)
        M = compose_affine_maps(prev_map, M)
        prev_map = M
        
        dst = cv.warpAffine(img2_full, M, (img2_full.shape[1], img2_full.shape[0]))
        cv.imwrite(savedir + f'lumen_affine_{i+1}.tif', dst)
        print(i)

/usr/local/lib/python3.8/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/usr/local/lib/python3.8/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/usr/local/lib/python3.8/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/usr/local/lib/python3.8/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)


167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226


[ WARN:0@3250.169] global /mnt/onetb/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('/mnt/onetb/yuuka/lumen/Lumen_228.tif'): can't open/read file: check file path/integrity


error: OpenCV(4.6.0-pre) /mnt/onetb/opencv/modules/imgproc/src/color.simd_helpers.hpp:92: error: (-2:Unspecified error) in function 'cv::impl::{anonymous}::CvtHelper<VScn, VDcn, VDepth, sizePolicy>::CvtHelper(cv::InputArray, cv::OutputArray, int) [with VScn = cv::impl::{anonymous}::Set<3, 4>; VDcn = cv::impl::{anonymous}::Set<1>; VDepth = cv::impl::{anonymous}::Set<0, 2, 5>; cv::impl::{anonymous}::SizePolicy sizePolicy = cv::impl::<unnamed>::NONE; cv::InputArray = const cv::_InputArray&; cv::OutputArray = const cv::_OutputArray&]'
> Invalid number of channels in input image:
>     'VScn::contains(scn)'
> where
>     'scn' is 1
